```{contents}
```
## Interrupt

An **Interrupt** in LangGraph is a **controlled pause in graph execution** that allows **external intervention** (usually by a human or an external system) before the workflow continues.
It is a core mechanism for building **human-in-the-loop, safe, auditable, and production-grade LLM systems**.

---

### **1. Why Interrupts Are Necessary**

Purely autonomous LLM workflows are unsafe for many domains.
Interrupts introduce **governance, oversight, and control**.

| Use Case   | Why Interrupt                |
| ---------- | ---------------------------- |
| Legal      | Mandatory human approval     |
| Finance    | Prevent unauthorized actions |
| Medical    | Clinical review              |
| Operations | Safety validation            |
| AI agents  | Prevent runaway behavior     |

---

### **2. Conceptual Model**

```
Node A → Node B → [ INTERRUPT ] → (External Input) → Resume → Node C
```

Execution state is **checkpointed** at the interrupt.

---

### **3. How Interrupt Works Internally**

1. Graph reaches an interrupt node
2. Execution **pauses**
3. State is **persisted**
4. Control is returned to the caller
5. External input modifies the state
6. Graph **resumes from the same node**

---

### **4. Basic Interrupt Example**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    text: str
    approved: bool

def analyze(state):
    return {"text": state["text"].upper()}

def wait_for_approval(state):
    from langgraph.pregel import interrupt
    response = interrupt({"text": state["text"]})
    return {"approved": response["approved"]}

builder = StateGraph(State)
builder.add_node("analyze", analyze)
builder.add_node("approve", wait_for_approval)

builder.set_entry_point("analyze")
builder.add_edge("analyze", "approve")
builder.add_conditional_edges("approve", lambda s: END if s["approved"] else "approve", {
    "approve": "approve",
    END: END
})

graph = builder.compile()
```

---

### **5. Running with Interrupt**

```python
result = graph.invoke({"text": "contract terms", "approved": False})
```

Execution **halts** and returns:

```python
{'text': 'CONTRACT TERMS'}
```

Resume:

```python
graph.invoke({"approved": True}, config={"thread_id": result["thread_id"]})
```

---

### **6. Interrupt vs Conditional Routing**

| Feature         | Interrupt | Conditional Edge |
| --------------- | --------- | ---------------- |
| Pause execution | Yes       | No               |
| External input  | Required  | Not required     |
| State persisted | Yes       | No               |
| Human control   | Yes       | No               |
| Resumable       | Yes       | No               |

---

### **7. Common Interrupt Patterns**

| Pattern             | Purpose               |
| ------------------- | --------------------- |
| Approval Gate       | Require authorization |
| Human Review        | Correct LLM output    |
| Safety Check        | Block unsafe actions  |
| Compliance Sign-off | Regulatory validation |
| Escalation          | Supervisor decision   |

---

### **8. Production-Grade Capabilities Enabled by Interrupts**

* Long-running workflows
* Fault tolerance
* Safe agent deployment
* Multi-step approvals
* Full auditability
* External system coordination

---

### **9. Best Practices**

* Place interrupts before irreversible actions
* Always checkpoint at interrupts
* Limit recursive resume loops
* Log every interrupt event
* Secure interrupt channels

---

### **10. Mental Model**

> **Interrupt = Controlled yield of execution with persistent state**

This makes LangGraph suitable for **regulated, safe, and interactive AI systems**.


In [9]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import interrupt
class State(TypedDict):
    input: str
    approved: bool

def analyze(state):
    print("Analyzing:", state["input"])
    return {}

def request_approval(state):
    # Interrupt execution and wait for external input
    
    raise interrupt("Waiting for human approval...")

def finalize(state):
    print("Finalizing...")
    return {"approved": True}


In [10]:
builder = StateGraph(State)

builder.add_node("analyze", analyze)
builder.add_node("approval", request_approval)
builder.add_node("finalize", finalize)

builder.set_entry_point("analyze")
builder.add_edge("analyze", "approval")
builder.add_edge("approval", "finalize")
builder.add_edge("finalize", END)

graph = builder.compile()


In [11]:
try:
    graph.invoke({"input": "Deploy model"})
except Exception as e:
    print(e)


Analyzing: Deploy model
'module' object is not callable


In [17]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import interrupt

class State(TypedDict):
    input: str
    approved: bool

def analyze(state):
    print("Analyzing:", state["input"])
    return {}

def request_approval(state):
    # Pause execution and wait for external input
    response = interrupt({"message": "Waiting for human approval"})
    return {"approved": response.get("approved", False)}

def finalize(state):
    print("Finalizing...")
    return {"approved": True}

builder = StateGraph(State)

builder.add_node("analyze", analyze)
builder.add_node("approval", request_approval)
builder.add_node("finalize", finalize)

builder.set_entry_point("analyze")
builder.add_edge("analyze", "approval")
builder.add_edge("approval", "finalize")
builder.add_edge("finalize", END)

# Enable interrupts + persistence
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [19]:
result = graph.invoke(
    {"input": "Deploy model"},
    config={"thread_id": "session-1"}
)
print(result)

Analyzing: Deploy model
{'input': 'Deploy model', '__interrupt__': [Interrupt(value={'message': 'Waiting for human approval'}, id='66321d40fd1788e3c17e570d8f5b336c')]}
